In [ ]:
# Librerías
import pandas as pd
import re
import unicodedata

In [ ]:
# Función general para limpiar el CSV
def clean_csv(file_path):
    # Cargar el CSV
    df = pd.read_csv(file_path, encoding='utf-8')

    # Convertir cadenas de texto a minúsculas y eliminar espacios
    for col in df.select_dtypes(include='object').columns:
        df[col] = df[col].str.lower().str.strip()

    # Eliminar duplicados
    df = df.drop_duplicates()

    # Reemplazar guiones con espacios en columnas de texto
    df = df.replace('-', ' ', regex=True)

    df = df.replace(["0", 0], pd.NA).dropna()

    return df

# Diccionario de palabras comunes para corregir
correcciones = {
    "nino": "niño",
    "nina":"niña",
    "ninos":"niños",
    "ninas":"niñas",
    "algodon": "algodón",
    "basico":"básico",
}

def normalize_text(text):
    if isinstance(text, str):
        # Normalizar acentos
        text = unicodedata.normalize('NFC', text)
        # Convertir a minúsculas
        text = text.lower()
        # Eliminar caracteres no deseados
        text = re.sub(r'[^\w\sáéíóúñü]', '', text)
        # Reemplazar palabras incorrectas usando el diccionario
        for palabra_erronea, palabra_correcta in correcciones.items():
            text = text.replace(palabra_erronea, palabra_correcta)
    return text


# Función para limpiar la columna específica 'Item'
def clean_item_column(df, columna):
    colores = [
        'rojo', 'verde', 'azul', 'amarillo', 'negro', 'blanco', 'gris', 'rosa',
        'morado', 'naranja', 'brown', 'black', 'white', 'yellow', 'green', 'blue',
        'purple', 'pink', 'turquesa','marron','beige','celeste'
    ]
    colores_pattern = r'\b(?:' + '|'.join(colores) + r'|color)\b'

    # Limpiar la columna
    df[columna] = df[columna].apply(lambda x: re.sub(colores_pattern, '', str(x))if pd.notna(x) else x)
    df[columna] = df[columna].apply(lambda x: re.sub(r'\b[A-Za-z0-9]{2,5}\d{1,4}\b', '', str(x)))
    df[columna] = df[columna].apply(lambda x: re.sub(r'\b(s|m|l|xl|xxl|2xl|xs|xxs|xxxl|tamaño)\b', '', str(x)))
    return df

 #Función para dividir precios de esta forma 35  s/45
def dividir_precios(df, precio_col):
    nuevas_filas = []  # Lista para almacenar las filas procesadas

    # Iterar sobre las filas originales
    for _, row in df.iterrows():
        precio = row[precio_col]

        # Verificar si el precio tiene el formato esperado
        if isinstance(precio, str) and 's/' in precio:
            # Limpieza inicial: eliminar espacios excesivos
            precio = precio.replace('   ', ' ').strip()

            # Extraer precios: soporte para decimales y prefijos opcionales
            precios = re.findall(r'\d+(?:\.\d+)?', precio)

            if len(precios) == 2:  # Si hay dos precios, dividirlos
                # Crear la fila original con el primer precio
                fila_original = row.copy()
                fila_original[precio_col] = precios[0]
                nuevas_filas.append(fila_original)

                # Crear una nueva fila con el segundo precio
                nueva_fila = row.copy()
                nueva_fila[precio_col] = precios[1]
                nuevas_filas.append(nueva_fila)

                continue  # Pasar a la siguiente fila después de dividir

        # Si no hay división, agregar la fila original
        nuevas_filas.append(row)

    # Crear un nuevo DataFrame con todas las filas procesadas
    return pd.DataFrame(nuevas_filas)

# Lista de rutas de los archivos
csv_files = ["oeschle.csv", "Saga_Falabella.csv", "Ripley.csv"]

# Procesar y guardar los archivos limpios
cleaned_dfs = {}
for file in csv_files:
    cleaned_df = clean_csv(file)

    # Aplicar normalización de texto y limpieza

    if 'Item' in cleaned_df.columns:
        cleaned_df['Item'] = cleaned_df['Item'].apply(normalize_text)
        cleaned_df = clean_item_column(cleaned_df, 'Item')
    if 'Categoria' in cleaned_df.columns:
           cleaned_df['Categoria'] = cleaned_df['Categoria'].apply(normalize_text)
    if 'Precio' in cleaned_df.columns:
        cleaned_df = dividir_precios(cleaned_df, 'Precio')
        #Asegurar que la columna Precio sea númerico
        cleaned_df['Precio'] = cleaned_df['Precio'].str.replace(',', '', regex=False)
        cleaned_df['Precio'] = pd.to_numeric(cleaned_df['Precio'], errors='coerce')
            # Eliminar filas donde "Precio" sea igual a 0
        cleaned_df = cleaned_df[cleaned_df["Precio"] != 0]

    #Guardar
    cleaned_dfs[file] = cleaned_df

for file_name, df in cleaned_dfs.items():
    print(f"\nArchivo procesado: {file_name}")
    files = f"cleaned_{file_name}"
    # Guardar cada data frame en un archivo csv
    df.to_csv(files, index=False,encoding='utf-8')
    print(f"Archivo guardado:{files}")



Archivo procesado: oeschle.csv
Archivo guardado:cleaned_oeschle.csv

Archivo procesado: Saga_Falabella.csv
Archivo guardado:cleaned_Saga_Falabella.csv

Archivo procesado: Ripley.csv
Archivo guardado:cleaned_Ripley.csv


In [ ]:

# Leer el archivo CSV
df = pd.read_csv('/content/muestra_unida.csv')

# Especificar las categorías a eliminar
categorias_a_eliminar = [
    "gorras y chullos",
    "ropas de baño para mujer",
    "corbatas y michis",
    "pijamas",
    "pijamas para hombre"
]

# Contabilizar los registros antes de la eliminación
registros_iniciales = len(df)

# Filtrar los registros que NO contengan las categorías a eliminar
df_filtrado = df[~df['Categoria'].isin(categorias_a_eliminar)]

# Filtrar los registros que NO tengan 'ropa de baño' en la columna 'Item' (eliminando esos registros)
df_filtrado = df_filtrado[~df_filtrado['Item'].str.contains('ropa de baño', case=False, na=False)]

# Filtrar los registros que NO tengan 'ropa de baño' en la columna 'Item' (eliminando esos registros)
df_filtrado = df_filtrado[~df_filtrado['Item'].str.contains('bikini', case=False, na=False)]

# Filtrar los registros que NO tengan 'ropa de baño' en la columna 'Item' (eliminando esos registros)
df_filtrado = df_filtrado[~df_filtrado['Item'].str.contains('trikini', case=False, na=False)]

# Contabilizar los registros eliminados
registros_eliminados = registros_iniciales - len(df_filtrado)

print(df_filtrado['Categoria'].unique())

# Guardar el DataFrame filtrado en un nuevo archivo CSV
df_filtrado.to_csv('muestra_unida_borrar.csv', index=False)

# Imprimir los resultados

print(f"Registros eliminados: {registros_eliminados}")
print(f"Cantidad de registros restantes: {len(df_filtrado)}")

# Leer el archivo CSV original
df = pd.read_csv('muestra_unida_borrar.csv')

# Sinónimos de cada género
sinonimos_hombre = ['hombre', 'varón', 'varon', 'caballero', 'señor']
sinonimos_mujer = ['mujer', 'muchacha', 'señorita', 'señora', 'dama', 'femenina']
sinonimos_nino = ['niño', 'niños', 'varoncito']
sinonimos_nina = ['niña', 'niñas','mujercita']
sinonimos_bebe = ['bebe', 'bebé']
sinonimos_unisex = ['unisex']


# Función para asignar el género basado en las palabras clave en Categoria y Item
def asignar_genero(row):
    categoria = row['Categoria'].lower()
    item = row['Item'].lower()

    # Revisar en Item si contiene algún sinónimo de género
    if any(s in item for s in sinonimos_hombre):
        return 'hombre'
    if any(s in item for s in sinonimos_mujer):
        return 'mujer'
    if any(s in item for s in sinonimos_nino):
        return 'niño'
    if any(s in item for s in sinonimos_nina):
        return 'niña'
    if any(s in item for s in sinonimos_bebe):
        return 'bebe'
    if any(s in item for s in sinonimos_unisex):
        return 'sin genero'

    # Si no se encuentra en Item, revisar en Categoria
    if any(s in categoria for s in sinonimos_hombre):
        return 'hombre'
    if any(s in categoria for s in sinonimos_mujer):
        return 'mujer'
    if any(s in categoria for s in sinonimos_nino):
        return 'niño'
    if any(s in categoria for s in sinonimos_nina):
        return 'niña'
    if any(s in categoria for s in sinonimos_bebe):
        return 'bebe'

    # Asignaciones adicionales por categorías específicas (ejemplo lencería o bebé)
    if 'lenceria' in categoria or 'calzon' in categoria or 'sostenes' in categoria or 'sandalias' in categoria or 'fajas' in categoria or 'botas' in categoria or 'botines' in categoria or 'botas' or categoria in 'pantys':
        return 'mujer'
    if 'ropa de bebe' in categoria or 'bebé' in categoria:
        return 'bebe'
    if 'pantuflas hombre' in categoria or 'mocasines' in categoria or 'gorros' in categoria:
        return 'bebe'

    # Si no se encuentra ningún género, asignar 'no asignado'
    return 'sin genero'

# Crear la nueva columna 'genero' con la asignación
df['genero'] = df.apply(asignar_genero, axis=1)

# Contabilizar los géneros
contabilizado = df['genero'].value_counts()

# Guardar el DataFrame filtrado y clasificado en un nuevo archivo CSV
df.to_csv('muestra_unida_genero.csv', index=False)

# Imprimir el resultado
print("Contabilización de géneros:")
print(contabilizado)

# Mostrar las primeras filas del DataFrame con la columna 'genero'
print("\nPrimeras filas del DataFrame clasificado:")
print(df.head())